In [1]:
from data import CollaborativeGPTGeneratorBatch,CollaborativeGPTGeneratorTestBatch
from tokenizer import DynamicBPETokenizerBatch, DynamicSPTokenizerBatch
import os
from data import ITEM_CONTENT_FILES
from model import UserItemMemory
from torch.utils.data import DataLoader
from model import DynamicCollaborativeGPTwithItemLMHeadBatch, DynamicDPELLM4RecBaseModel,MSEDynamicDPELLM4RecBaseModel
from transformers import GPT2Model, GPT2Config, AutoModel, AutoModelForSequenceClassification

import json
import fsspec


c:\Users\vulam\miniconda3\envs\recsys\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# Load train_mat
from scipy.sparse import load_npz
train_mat = load_npz(r"C:\Users\vulam\Downloads\Master_papers\code\cold_item\MI4Rec\dataset\beauty\warm_train_matrix.npz")
test_mat = load_npz(r"C:\Users\vulam\Downloads\Master_papers\code\cold_item\MI4Rec\dataset\beauty\warm_test_matrix.npz")


In [3]:
# Load necessary things for tokenizer
tokenizer_root = r"C:\Users\vulam\Downloads\Master_papers\code\cold_item\MI4Rec\model\pretrained\gpt2"
vocab_file = os.path.join(tokenizer_root, "vocab.json")
merges_file = os.path.join(tokenizer_root, "merges.txt")
num_users = train_mat.shape[0]
num_items = train_mat.shape[1]
memory = UserItemMemory()
data_root = r"C:\Users\vulam\Downloads\Master_papers\code\cold_item\MI4Rec\dataset\beauty"

In [4]:
# Load tokenizer
tokenizer = DynamicBPETokenizerBatch(vocab_file,
            merges_file,
            num_users,
            num_items,
            memory,
            [],
            [os.path.join(data_root, "item_texts", file) for file in ITEM_CONTENT_FILES]
        )

In [22]:
collaborative_data_gen = CollaborativeGPTGeneratorTestBatch(tokenizer, test_mat)

In [23]:
test_dataloader = DataLoader(collaborative_data_gen, 
                              batch_size = 4,
                              collate_fn=collaborative_data_gen.collate_fn)

In [7]:
_config_path =  os.path.join(r"C:\Users\vulam\Downloads\Master_papers\code\cold_item\MI4Rec\model\pretrained\gpt2\config.json")
with fsspec.open(_config_path, "r") as f:
        _config = json.load(f)
config = GPT2Config(**_config)
config.num_users = num_users
config.num_items = num_items

# LLMModel
LLMmodel = GPT2Model(config)
LLMmodel.to("cuda")

# device 
device = "cuda"
num_meta = 300
prob_norm = "softmax"

In [8]:
collaborative_base_model = MSEDynamicDPELLM4RecBaseModel(
        config, 
        LLMmodel, 
        memory, 
        None, # TODO: use local tokenizer
        None, 
        device=device,
        num_item_meta=num_meta,
        item_logits_infer='original',
        prob_norm=prob_norm,
    )
collaborative_base_model.to(device)

MSEDynamicDPELLM4RecBaseModel(
  (user_embeddings): Embedding(22363, 768)
  (LLMmodel): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (item_embeddings): Embedding(12101, 768)
  (meta_item_embeddings): E

In [9]:
# load _model 
collaborative_model = DynamicCollaborativeGPTwithItemLMHeadBatch(config, base_model = collaborative_base_model,device=device)
collaborative_model.to(device)

DynamicCollaborativeGPTwithItemLMHeadBatch(
  (base_model): MSEDynamicDPELLM4RecBaseModel(
    (user_embeddings): Embedding(22363, 768)
    (LLMmodel): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D(nf=2304, nx=768)
            (c_proj): Conv1D(nf=768, nx=768)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=3072, nx=768)
            (c_proj): Conv1D(nf=768, nx=3072)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), ep

In [15]:
import numpy as np
def Recall_at_k(y_true, y_pred, k, agg="sum"):
    '''
        Average recall for top k recommended results.
        The training records should be set to -inf in y_pred
    '''
    y_true_bin = (y_true > 0)
    true_positives_count = np.sum(y_true_bin, axis=1)
    batch_size = y_pred.shape[0]
    topk_idxes = np.argpartition(-y_pred, k, axis=1)[:, :k]
    y_pred_bin = np.zeros_like(y_pred, dtype=bool)
    y_pred_bin[np.arange(batch_size)[:, None], topk_idxes] = True
    y_true_bin = (y_true > 0)
    true_positives_count = np.sum(y_true_bin, axis=1)
    valid_mask = true_positives_count > 0
    hits = np.sum(np.logical_and(y_true_bin, y_pred_bin), axis=-1).astype(np.float32)
    hits = hits[valid_mask]
    # print(np.sum(y_true_bin, axis=1), hits)
    recalls = hits/np.minimum(k, np.sum(y_true_bin, axis=1)[valid_mask])
    if agg == "sum":
        recall = np.sum(recalls)
    elif agg == "mean":
        recall = np.mean(recalls)
    else:
        raise NotImplementedError(f"aggregation method {agg} not defined!")
    return recall

In [ ]:
from tqdm import tqdm
collaborative_model.eval()
recall_overall = 0
count = 0 
for input_ids_prompt, input_ids_main, target_matricies, attention_mask in tqdm(test_dataloader):
    input_ids_prompt = input_ids_prompt.to(device)
    input_ids_main = input_ids_main.to(device)
    attention_mask = attention_mask.to(device)

    try:
        outputs, scores = collaborative_model(input_ids_prompt, 
                                    input_ids_main,
                                    labels_main=input_ids_main,
                                    attention_mask=attention_mask,
                                    eval = True)
    except:
        count += 1
        print(count)
        continue

    
    
    recall_overall += Recall_at_k(target_matricies.detach().numpy(), scores.cpu().detach().numpy(), k=20, agg="sum")

print(recall_overall / len(test_dataloader))


    

  1%|          | 36/5591 [00:02<04:11, 22.06it/s]

1


  2%|▏         | 120/5591 [00:05<03:49, 23.88it/s]

2
3


  3%|▎         | 153/5591 [00:07<03:52, 23.38it/s]

...

 59%|█████▊    | 3278/5591 [8:59:08<04:28,  8.63it/s]

236


 59%|█████▉    | 3288/5591 [8:59:10<04:40,  8.21it/s]

237


 59%|█████▉    | 3293/5591 [8:59:10<04:12,  9.10it/s]

In [ ]:
a.shape

torch.Size([4, 4, 768])

In [ ]:
a.mean(axis = 1).shape

torch.Size([4, 768])

In [ ]:
import torch
b = torch.Tensor(
    [[
        [1,2,3,7],
        [4,5,6,8],
        [4,5,6,9]
    ],
    [
        [5,6,7,1],
        [6,7,8,2],
        [4,5,6,9]
    ]
    ]
)

In [ ]:
b.mean(axis = 1).shape

torch.Size([2, 4])

In [ ]:
for i in range(10):
    print(11)
    if i == 10:
        print(f"That i

torch.Size([2, 3, 4])